In [1]:
import librosa
import os
import numpy as np
import pandas as pd
import soundfile as sf

In [2]:
AUDIO_DATA_PATH = 'data/'
GPATH = '/content/drive/My Drive/Master/Asignaturas/2 Cuatrimestre/Proyecto/Code/'
LPATH_DATA = 'SpeechEmotionRecognition/data/processed/'
LPATH_AUGMENTED = GPATH + AUDIO_DATA_PATH + 'RAVDESS/augmented'
SAMPLE_FILE = "03-01-01-01-01-01-01.wav"

# Maps
EMOTION_MAP = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
INTENSITY_MAP = {1:'normal', 2:'strong'}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dir_list = os.listdir(GPATH + AUDIO_DATA_PATH)
dir_list.sort()

print(dir_list)

emotion = []
gender = []
intensity = []
path = []

# Extraemos de cada archivo de sonido sus datos
for dir in dir_list:
  path_dir = os.listdir(GPATH + AUDIO_DATA_PATH + dir) # todos los archivos de audios asociados a un directorio
  for filepath in path_dir:
    info_vector = filepath.split('.')[0].split('-')
    n_emotion = int(info_vector[2])
    n_gender = int(info_vector[6])
    n_intensity = int(info_vector[3])
    str_path = GPATH + AUDIO_DATA_PATH + dir + '/' + str(filepath)
    path.append(str_path)
    emotion.append(n_emotion)
    intensity.append(n_intensity)
    if n_gender%2 == 0:
      gender.append('female')
    else:
      gender.append('male')

# Construimos el data frame
EnglishSpeech_df = pd.DataFrame(columns=['emotion', 'gender', 'intensity', 'path'])
EnglishSpeech_df['emotion'] = emotion
EnglishSpeech_df['gender'] = gender
EnglishSpeech_df['intensity'] = intensity
EnglishSpeech_df['path'] = path
EnglishSpeech_df['emotion'] = EnglishSpeech_df['emotion'].map(EMOTION_MAP) 
EnglishSpeech_df['intensity'] = EnglishSpeech_df['intensity'].map(INTENSITY_MAP)


print("Size of the dataset: {} \n".format(len(EnglishSpeech_df)))
class_distribution = EnglishSpeech_df['emotion'].value_counts()
print(class_distribution)

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24', 'RAVDESS']
Size of the dataset: 1440 

angry       192
happy       192
surprise    192
fear        192
sad         192
calm        192
disgust     192
neutral      96
Name: emotion, dtype: int64


### **GENERACION DE LOS DATOS AUMENTADOS**

In [13]:
def load_ravdess_dataAugmentation(df, augmented_path='', pitch = True, stretch = True, step_list = {1}, rate_list = {1}, verbose = False):
  '''
  Genera archivos de audio basado en las modificaciones que se especifican en los parametros. 
  La informacion de las pistas de audio se le pasa al funcion mediante un dataframe.
  '''
  for index, row in df.iterrows():
    # Data augmentation basado en desplazar el tono
    if pitch:
      save_augmented_pitch(row['path'], step_list, augmented_path + '/' + row['emotion'], index,  verbose)
      if verbose:
        print("Generadas {} muestras de {}".format(len(step_list), row['emotion']) )
    # Data augmentation basado en stretching
    if stretch:
      save_augmented_stretch(row['path'], rate_list, augmented_path + '/' + row['emotion'], index, verbose)
      if verbose:
        print("Generadas {} muestras de {}".format(len(rate_list), row['emotion']))


def save_augmented_pitch(dirdata, step_list, outpath, index, verbose = False):
  '''
  Modifica desplazando el tono una pista de audio y la guarda. Esta funcion 
  guarda las pistas generadas creando una carpeta con el nombre de la clase 
  a la que pertenece. La modificacion de dicho tono se basa en una lista dada 
  en los parametros
  Arguments
  ---------
    dirdata: str
    step_list: list
    outpath: str
  Return
  ---------
    void
  '''
  for i, n_steps in enumerate(step_list):
    y, sr = librosa.load(dirdata, duration=2.97)
    y_changed_pitch = librosa.effects.pitch_shift(y, sr, n_steps=n_steps)
    # outpath = LPATH_AUGMENTED + '/' + row['emotion'] 
    if not os.path.exists(outpath):
      os.makedirs(outpath)
    filename = outpath + '/augmented_pitch_0'+ str(i) + '_0' + str(index) + '.wav'
    sf.write( filename ,y_changed_pitch, sr)


def save_augmented_stretch(dirdata, rate_list, outpath, index, verbose = False):
  '''
  Modifica una pista de audio estirando el tiempo un rango fijado. Dicho rango 
  (o rangos) se especifica en una lista como parametro. Esta funcion 
  guarda las pistas generadas creando una carpeta con el nombre de la clase 
  a la que pertenece
  Arguments
  ---------
    dirdata: str
    step_list: list
    outpath: str
  Return
  ---------
    void

  '''
  for i, rate in enumerate(rate_list):
    y, sr = librosa.load(dirdata , duration=2.97)
    y_changed_shitf = librosa.effects.time_stretch(y, rate=rate)
    # augmented_path = LPATH_AUGMENTED + '/' + row['emotion'] 
    if not os.path.exists(outpath):
      os.makedirs(outpath)
    filename = outpath + '/augmented_stretch_0'+ str(i) + '_0' + str(index) + '.wav'
    sf.write( filename ,y_changed_shitf, sr)

In [14]:
load_ravdess_dataAugmentation(EnglishSpeech_df, LPATH_AUGMENTED, 
                              pitch = True, 
                              stretch =True, 
                              step_list = {-1, -2, -2.5, -3.5, 1 ,2 ,2.5, 3.5}, 
                              rate_list = {0.81, 1.07},
                              verbose = True)

Generadas 8 muestras de neutral
Generadas 2 muestras de neutral
Generadas 8 muestras de calm
Generadas 2 muestras de calm
Generadas 8 muestras de neutral
Generadas 2 muestras de neutral
Generadas 8 muestras de calm
Generadas 2 muestras de calm
Generadas 8 muestras de calm
Generadas 2 muestras de calm
Generadas 8 muestras de neutral
Generadas 2 muestras de neutral
Generadas 8 muestras de calm
Generadas 2 muestras de calm
Generadas 8 muestras de neutral
Generadas 2 muestras de neutral
Generadas 8 muestras de calm
Generadas 2 muestras de calm
Generadas 8 muestras de calm
Generadas 2 muestras de calm
Generadas 8 muestras de sad
Generadas 2 muestras de sad
Generadas 8 muestras de calm
Generadas 2 muestras de calm
Generadas 8 muestras de happy
Generadas 2 muestras de happy
Generadas 8 muestras de sad
Generadas 2 muestras de sad
Generadas 8 muestras de happy
Generadas 2 muestras de happy
Generadas 8 muestras de sad
Generadas 2 muestras de sad
Generadas 8 muestras de happy
Generadas 2 muestras

### **H**